# Import Dependencies

In [ ]:
#!apt-get -y install python3
# !curl https://bootstrap.pypa.io/get-pip.py -o get-pip.py
# !sudo python3 get-pip.py
# !python3 -m pip install ipykernel
# !python3 -m ipykernel install --user

In [ ]:
#!pip3 install -r requirements.txt

In [6]:
from datagen_customer import Customer, Headers
from demographics import make_cities, make_age_gender_dict
from faker import Faker
import pandas as pd
import numpy as np
import datetime
import random
import google.auth
from google.cloud import storage
import os
import pandas_gbq
from os import listdir
from os.path import isfile, join
from datetime import date
import faker
import profile_weights
from datagen_transaction import T_Customer, create_header
import os
import multiprocessing as mp
#print("Number of processors: ", mp.cpu_count())
from google.cloud import storage

In [7]:
#service account needs write access into bq project
# os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/Users/leighaj/Service Accounts/looker-private-demo-d4bcf655c1aa.json'

credentials, project = google.auth.default(
    scopes=[
        "https://www.googleapis.com/auth/bigquery"
    ]
)

In [15]:
#set random seed for faker
fake = Faker()
Faker.seed(44)

# Create New Customers
### Skip if you already have a customers.csv

In [ ]:
#create 100,000 customers which are saved to file
for _ in range(100000):
    Customer(fake = fake, filename = 'customers.csv')

In [ ]:
#assign a traffic source
def create_traffic_source(profile):
    traffic_sources = ['Mailer','Email','Call Center','Website','Third Party Advertisement']
    if 'young_adults' in profile:
        #more likely to be third party adverisement or website
        traffic_sources = traffic_sources + [traffic_sources[4],traffic_sources[4],traffic_sources[4
                                    ],traffic_sources[3],traffic_sources[3]]
    elif 'urban' in profile:
        traffic_sources = traffic_sources + [traffic_sources[1],traffic_sources[2],traffic_sources[2
                        ],traffic_sources[3],traffic_sources[3]]
    else:
        traffic_sources = traffic_sources + [traffic_sources[1],traffic_sources[1],traffic_sources[1
                ],traffic_sources[2],traffic_sources[3]]
    
    return np.random.choice(traffic_sources)

In [ ]:
#read in csv as a dataframe
clients = pd.read_csv('customers.csv','|',names=['ssn', 'cc_num', 'first', 'last', 'gender', 'street', \
                    'city', 'state', 'zip', 'lat', 'long', 'city_pop', \
                    'job', 'dob', 'acct_num', 'profile'])

In [ ]:
clients.head()

In [ ]:
## already exists
#ssn
#first_name
#last_name
#gender
#street
#address
#job
#profile
#dob 

## need to create
#age - create in database
#district_id - keep as null
#client_id - keep as null

#client_id add row number 
clients = clients.reset_index()
clients['disp_id'] = clients['index'].apply(lambda x: x + 13661)
clients['card_id'] = clients['index'].apply(lambda x: x + 1462)

#traffic_source
clients['traffic_source'] = clients['profile'].apply(lambda x: create_traffic_source(x))

In [ ]:
clients = clients.sample(frac=1)
clients = clients.drop(['index'], axis=1)
clients = clients.reset_index()

clients['card_type'] = None
clients['card_start'] = None
clients['card_end'] = None

In [ ]:
#Give 70% of young adults junior card, 20% junior2 card and 6% classic, 3% classic2 and rest gold
length = len(clients)

clients.loc[clients['profile'].str.contains('young'),'card_type'] = clients.loc[
    clients['profile'].str.contains('young')].apply(lambda x: 'Junior' if int(x['index']) < .7*length 
            else 'Junior2' if int(x['index']) <.9*length 
            else 'Classic' if int(x['index']) <.96*length
            else 'Classic2' if int(x['index']) <.99*length else 'Gold', axis=1)

In [ ]:
#Give 50% of classic, 35% classic2, 15% gold 
#length = len(clients.loc[clients['profile'].str.contains('50up')])

clients.loc[clients['profile'].str.contains('50up'),'card_type'] = clients.loc[
    clients['profile'].str.contains('50up')].apply(lambda x: 'Classic' if int(x['index']) < .5*length 
            else 'Classic2' if int(x['index']) <.35*length
            else 'Gold', axis=1)

In [ ]:
#Give 60% of classic, 30% classic2, 10% gold 
#length = len(clients.loc[clients['profile'].str.contains('2550')])

clients.loc[clients['profile'].str.contains('2550'),'card_type'] = clients.loc[
    clients['profile'].str.contains('2550')].apply(lambda x: 'Classic' if int(x['index']) < .6*length 
            else 'Classic2' if int(x['index']) <.9*length 
            else 'Gold', axis=1)

In [ ]:
# randomly chose a start date that is after november 2015 more than 15 years after their birthday, and before end of this year

clients['card_start'] = clients['dob'].apply(lambda x: 
        datetime.timedelta(days=random.randint(max(15*365,(pd.to_datetime('2015-11-01')-pd.to_datetime(
                        x)).days), (pd.to_datetime(
        '2020-12-31')-pd.to_datetime(x)).days)) + pd.to_datetime(x))

In [ ]:
# for junior cards - take 80% of cards and create an end date and a new card add date
if 'index' in clients.columns:
    clients = clients.drop(['index'], axis=1)
results = clients.loc[clients['card_type'] == 'Junior'].sample(frac=.9).reset_index()


#randomly generate a normal distribution of time they convert to next card, and the time they take to cancel previous
days_to_add = list(np.random.normal(365, 150, len(results)))
days_to_cancel = list(np.random.normal(30*3, 25, len(results)))
new_card_id = clients['card_id'].max()

new_cards = []

for i in range(len(results)):
    card_id = results.loc[i,'card_id']
    start_date = clients.loc[clients['card_id'] == card_id,'card_start']
    new_card_start_date = pd.to_datetime(start_date.values[0]) + datetime.timedelta(days=days_to_add.pop())
    cancel_date = new_card_start_date + datetime.timedelta(days=days_to_cancel.pop())
    
    
    #update the end date
    clients.loc[clients['card_id'] == card_id,'card_end'] = cancel_date
    
    
    #15% just cancel
    if i < .85*len(results) and new_card_start_date <= pd.to_datetime('2020-12-31'):
        #55% go to Junior2
        #30% go to Classic
        if i<.55*len(results):
            new_card_type = 'Junior2'
        else:
            new_card_type = 'Classic'
        
        new_card_id += 1 
        new_card = clients.loc[clients['card_id'] == card_id].to_dict('records')[0]
        new_card['card_id'] = new_card_id
        new_card['card_type'] = new_card_type
        new_card['card_start'] = new_card_start_date
        new_card['card_end'] = np.nan
        new_card['cc_num'] = fake.credit_card_number()
        new_cards.append(new_card)
    
clients = clients.append(pd.DataFrame(new_cards)) 

In [ ]:
clients = clients.loc[clients['card_start']<pd.to_datetime('2020-12-31')]

In [ ]:
#for each gold member
#change the original card_type 
#then randomly chose a date between now and when the original card was created to sign up for gold
#also if index is in randomly chosen 10% then chose an end date for the gold card also

results = clients.loc[clients['card_type'] == 'Gold'].sample(frac=1).reset_index()

ix_cl = round(len(results)*.6)
ix_j2 = ix_cl+round(len(results)*.1)
ix_cl2 = ix_j2+round(len(results)*.1)
rand_10 = random.sample(range(0, ix_cl2), round(ix_cl2*.25))


new_cards = []
new_card_id = max(clients['card_id'])
curr_ix = 0

days_to_add = list(np.random.normal(600, 150, len(results)))
days_to_cancel = list(np.random.normal(30*3, 25, len(results)))

for ix in [ix_cl,ix_j2,ix_cl2]:
    for i in range(curr_ix,ix):
        card_id = results.loc[i,'card_id']
        disp_id = clients.loc[clients['card_id'] == card_id,'disp_id'].values[0]
        if ix == ix_cl:
            card_type = 'Classic2'
        elif ix == ix_j2:
            card_type = 'Junior2'
        else:
            card_type = 'Classic'
        
        clients.loc[clients['card_id'] == card_id,'card_type'] = card_type
        
        
        created_date = pd.to_datetime(clients.loc[clients['card_id'] == card_id,'card_start'
                             ].values[0])
        new_card_date = created_date + datetime.timedelta(days=days_to_add.pop()) 
        new_cancel_date = np.nan
        
        if i in rand_10[0:15]:
            new_cancel_date = new_card_date + datetime.timedelta(days=days_to_cancel.pop()) 
        
        elif i in rand_10[15:]:
            if created_date != datetime.datetime(2020, 12, 31, 0, 0):
                clients.loc[clients['card_id'] == card_id,'card_end'] = created_date + datetime.timedelta(days=random.randrange((
                    datetime.datetime(2020, 12, 31, 0, 0) - created_date).days)) 

        
        #update cards
        new_card_id += 1 
        new_card = clients.loc[clients['card_id'] == card_id].to_dict('records')[0]
        new_card['card_id'] = new_card_id
        new_card['card_type'] = 'Gold'
        new_card['card_start'] = new_card_date
        new_card['card_end'] = new_cancel_date
        new_card['cc_num'] = fake.credit_card_number()
        new_cards.append(new_card)
        
    curr_ix = ix
clients = clients.append(pd.DataFrame(new_cards)) 

In [ ]:
#add clients staging
pandas_gbq.context.credentials = credentials
pandas_gbq.context.project = "looker-private-demo"
pd.DataFrame(clients).to_gbq('finserv_staging.new_clients_7_20',if_exists = 'fail', project_id=project)

# Create New Transactions
### If you already have a customers.csv file then you only need to run these cells

In [8]:
clients = pd.read_gbq('select * from finserv_staging.new_clients_7_20', project_id='looker-private-demo')
clients['card_start'] = clients['card_start'].dt.tz_localize(None)
clients['card_end'] = pd.to_datetime(clients['card_end']).dt.tz_localize(None)

/usr/local/lib/python3.5/dist-packages/pandas_gbq/gbq.py:562: UserWarning: A progress bar was requested, but there was an error loading the tqdm library. Please install tqdm to use the progress bar functionality.
  progress_bar_type=progress_bar_type,


In [9]:
profile_files = [f for f in listdir('profiles/'
                                   ) if isfile(join('profiles/', f))]
profile_files = [join('profiles/', f) 
                 for f in profile_files if f.split('_')[0] in ('adults','young')]

In [10]:
customers_file = open('customers.csv', 'r').readlines() 

In [11]:
headers = ''
for h in ['ssn', 'cc_num', 'first', 'last', 'gender', 'street', \
            'city', 'state', 'zip', 'lat', 'long', 'city_pop', \
            'job', 'dob', 'acct_num', 'profile']:
    headers += h + '|'
headers = headers[:-1]
headers = create_header(headers)

ssn|cc_num|first|last|gender|street|city|state|zip|lat|long|city_pop|job|dob|acct_num|profile|trans_num|trans_date|trans_time|unix_time|category|amt|is_fraud|merchant|merch_lat|merch_long


In [12]:
client = storage.Client()
bucket = client.get_bucket('retail_banking')

In [17]:
def base(a=0):
    counter = 0
    txns = []
    loop_counter = 0

    b=(a+1)*10000
    for i, customer_card in clients[(a*10000):b].iterrows():
        for m in profile_files:
            pro_name = m.split('profiles')[-1]
            pro_name = pro_name[1:]
            parse_index = m.index('profiles') + 9
            m_fraud = m[:parse_index] +'fraud_' + m[parse_index:]
            pro = open(m, 'r').read()
            pro_fraud = open(m_fraud, 'r').read()
            pro_name_fraud = 'fraud_' + pro_name

            curr_profile = pro_name
            curr_fraud_profile = pro_name_fraud
            profile_name = m

            card_id = customer_card['card_id']
            create_d = pd.to_datetime(customer_card['card_start'])

            end_d = pd.to_datetime(customer_card['card_end'])
            if end_d is None or pd.isnull(end_d):
                end_d = pd.to_datetime(date(2021,1,1))
                if (end_d-create_d).days < 10:
                    end_d = pd.to_datetime(date(2021,1,10))
            
            #print(create_d, end_d)
            profile = profile_weights.Profile(pro,create_d,end_d)
            cust = customers_file[[j for j, s in enumerate(customers_file) if customer_card['ssn'] in s][0]]
            t_cust = T_Customer(cust, profile, headers, fake)
            
            if t_cust.attrs['profile'] == curr_profile:
                is_fraud= 0
                fraud_flag = random.randint(0,100) 
                fraud_dates = []

                merch = pd.read_csv('demographic_data/merchants.csv' , sep='|')
                # decide if we generate fraud or not
                if fraud_flag < 15: #11->25
                    fraud_interval = random.randint(1,10) #7->1
                    
                    inter_val = (end_d-create_d).days-7
                    # rand_interval is the random no of days to be added to start date
                    if inter_val <= 0:
                        inter_val = 2
                    rand_interval = random.randint(1, inter_val)
                    #random start date is selected
                    newstart = create_d + datetime.timedelta(days=rand_interval)
                    # based on the fraud interval , random enddate is selected
                    newend = newstart + datetime.timedelta(days=fraud_interval)
                    print(newstart, newend)
                    # we assume that the fraud window can be between 1 to 7 days #7->1
                    profile = profile_weights.Profile(pro_fraud, newstart, newend)
                    t_cust = T_Customer(cust, profile, headers, fake)
                    is_fraud = 1
                    try:
                        temp_tx_data = profile.sample_from(is_fraud)
                        fraud_dates = temp_tx_data[3]
                        txns = t_cust.print_trans(temp_tx_data,is_fraud, fraud_dates,merch)
                    except:
                        pass
                    #parse_index = m.index('profiles/') + 9
                    #m = m[:parse_index] +'fraud_' + m[parse_index:]

                # we're done with fraud (or didn't do it) but still need regular transactions
                # we pass through our previously selected fraud dates (if any) to filter them
                # out of regular transactions
                profile = profile_weights.Profile(pro, create_d, end_d)
                is_fraud =0
                try:
                    temp_tx_data = profile.sample_from(is_fraud)
                except:
                    pass
                if txns is not None:
                    txns = txns + t_cust.print_trans(temp_tx_data, is_fraud, fraud_dates, merch)
                else:
                    txns = t_cust.print_trans(temp_tx_data, is_fraud, fraud_dates, merch)


                txns = pd.DataFrame(txns)
                #add in customer id
                txns['card_id'] = customer_card['card_id']

                #if the file isnt more than 5GB then append the transactions to it otherwise create new file
                filename = 'trans_thread_'+str(a)+'_'+str(counter)+'.csv'
                if os.path.exists(filename):
                    if os.path.getsize(filename)/1000000000 > 5:
                        #upload the file to gcp storage bucket
                        print('added to GCS: ', filename)
                        bucket.blob(filename).upload_from_filename(filename, 'text/csv')
                        counter += 1
                        filename = 'trans_thread_'+str(a)+'_'+str(counter)+'.csv'
                txns.to_csv(filename, mode='a', header=False, sep='|')
                
                loop_counter += 1
                
                
                print(a,'Thread, added ', len(txns),' to the table, for card ', card_id, ' card #: ', 
                      loop_counter, ' in file: ', filename)
                txns = None
    return ('done ', a)

In [ ]:
from multiprocessing import Process

procs = []
proc = Process(target=base) 
procs.append(proc)
proc.start()

# instantiating process with arguments
for a in range(12):
    proc = Process(target=base, args=(a,))
    procs.append(proc)
    proc.start()

# complete the processes
for proc in procs:
    proc.join()

2019-11-20 00:00:00 2019-11-30 00:00:00
0 Thread, added  1214  to the table, for card  47208  card #:  1  in file:  trans_thread_0_0.csv
0 Thread, added  2270  to the table, for card  47208  card #:  1  in file:  trans_thread_0_0.csv
2018-12-26 00:00:00 2019-01-02 00:00:00
9 Thread, added  250  to the table, for card  107941  card #:  1  in file:  trans_thread_9_0.csv
11 Thread, added  311  to the table, for card  95786  card #:  1  in file:  trans_thread_11_0.csv
6 Thread, added  3618  to the table, for card  51703  card #:  1  in file:  trans_thread_6_0.csv
0 Thread, added  60  to the table, for card  26609  card #:  2  in file:  trans_thread_0_0.csv
10 Thread, added  1322  to the table, for card  6998  card #:  1  in file:  trans_thread_10_0.csv
2019-12-24 00:00:00 2019-12-30 00:00:00
8 Thread, added  4900  to the table, for card  30026  card #:  1  in file:  trans_thread_8_0.csv
5 Thread, added  1690  to the table, for card  34106  card #:  1  in file:  trans_thread_5_0.csv
4 Threa

8 Thread, added  376  to the table, for card  78292  card #:  12  in file:  trans_thread_8_0.csv
2 Thread, added  9135  to the table, for card  51257  card #:  10  in file:  trans_thread_2_0.csv
3 Thread, added  2432  to the table, for card  111443  card #:  12  in file:  trans_thread_3_0.csv
7 Thread, added  230  to the table, for card  104338  card #:  12  in file:  trans_thread_7_0.csv
2020-11-15 00:00:00 2020-11-17 00:00:00
1 Thread, added  227  to the table, for card  64242  card #:  12  in file:  trans_thread_1_0.csv
6 Thread, added  1853  to the table, for card  57244  card #:  13  in file:  trans_thread_6_0.csv
0 Thread, added  2656  to the table, for card  42792  card #:  13  in file:  trans_thread_0_0.csv
9 Thread, added  1462  to the table, for card  80191  card #:  14  in file:  trans_thread_9_0.csv
10 Thread, added  0  to the table, for card  83103  card #:  15  in file:  trans_thread_10_0.csv
0 Thread, added  1811  to the table, for card  54444  card #:  14  in file:  tra

8 Thread, added  482  to the table, for card  80027  card #:  25  in file:  trans_thread_8_0.csv
6 Thread, added  8952  to the table, for card  52416  card #:  24  in file:  trans_thread_6_0.csv
10 Thread, added  2886  to the table, for card  86253  card #:  27  in file:  trans_thread_10_0.csv
0 Thread, added  2667  to the table, for card  6687  card #:  26  in file:  trans_thread_0_0.csv
9 Thread, added  1382  to the table, for card  46562  card #:  28  in file:  trans_thread_9_0.csv
11 Thread, added  0  to the table, for card  93299  card #:  27  in file:  trans_thread_11_0.csv
3 Thread, added  970  to the table, for card  76648  card #:  25  in file:  trans_thread_3_0.csv
4 Thread, added  3356  to the table, for card  6077  card #:  25  in file:  trans_thread_4_0.csv
5 Thread, added  1047  to the table, for card  40279  card #:  27  in file:  trans_thread_5_0.csv
6 Thread, added  985  to the table, for card  69650  card #:  25  in file:  trans_thread_6_0.csv
0 Thread, added  3630  t

10 Thread, added  2764  to the table, for card  60451  card #:  39  in file:  trans_thread_10_0.csv
11 Thread, added  1339  to the table, for card  9382  card #:  39  in file:  trans_thread_11_0.csv
3 Thread, added  216  to the table, for card  21984  card #:  37  in file:  trans_thread_3_0.csv
4 Thread, added  332  to the table, for card  116457  card #:  39  in file:  trans_thread_4_0.csv
5 Thread, added  596  to the table, for card  81752  card #:  39  in file:  trans_thread_5_0.csv
6 Thread, added  796  to the table, for card  82676  card #:  36  in file:  trans_thread_6_0.csv
0 Thread, added  156  to the table, for card  97011  card #:  36  in file:  trans_thread_0_0.csv
8 Thread, added  2810  to the table, for card  10832  card #:  37  in file:  trans_thread_8_0.csv
11 Thread, added  433  to the table, for card  19859  card #:  40  in file:  trans_thread_11_0.csv
0 Thread, added  2668  to the table, for card  112941  card #:  38  in file:  trans_thread_0_0.csv
2020-12-24 00:00:00

4 Thread, added  1570  to the table, for card  45515  card #:  52  in file:  trans_thread_4_0.csv
3 Thread, added  4941  to the table, for card  16050  card #:  49  in file:  trans_thread_3_0.csv
6 Thread, added  4869  to the table, for card  27101  card #:  46  in file:  trans_thread_6_0.csv
0 Thread, added  975  to the table, for card  24509  card #:  49  in file:  trans_thread_0_0.csv
10 Thread, added  0  to the table, for card  108732  card #:  54  in file:  trans_thread_10_0.csv
8 Thread, added  3988  to the table, for card  3763  card #:  49  in file:  trans_thread_8_0.csv
4 Thread, added  553  to the table, for card  103424  card #:  53  in file:  trans_thread_4_0.csv
9 Thread, added  1356  to the table, for card  57436  card #:  51  in file:  trans_thread_9_0.csv
2017-02-12 00:00:00 2017-02-19 00:00:00
5 Thread, added  1185  to the table, for card  34646  card #:  51  in file:  trans_thread_5_0.csv
2 Thread, added  2300  to the table, for card  9080  card #:  49  in file:  tran

...

5 Thread, added  2481  to the table, for card  50377  card #:  101  in file:  trans_thread_5_0.csv
0 Thread, added  1734  to the table, for card  99883  card #:  97  in file:  trans_thread_0_0.csv
0 Thread, added  2344  to the table, for card  74361  card #:  93  in file:  trans_thread_0_0.csv
7 Thread, added  3952  to the table, for card  59265  card #:  90  in file:  trans_thread_7_0.csv
6 Thread, added  4830  to the table, for card  41203  card #:  92  in file:  trans_thread_6_0.csv
10 Thread, added  5115  to the table, for card  85268  card #:  104  in file:  trans_thread_10_0.csv
8 Thread, added  7132  to the table, for card  69567  card #:  98  in file:  trans_thread_8_0.csv
2 Thread, added  3140  to the table, for card  73361  card #:  96  in file:  trans_thread_2_0.csv
4 Thread, added  1374  to the table, for card  114365  card #:  104  in file:  trans_thread_4_0.csv
1 Thread, added  7770  to the table, for card  28842  card #:  92  in file:  trans_thread_1_0.csv
3 Thread, adde

11 Thread, added  4476  to the table, for card  69538  card #:  108  in file:  trans_thread_11_0.csv
0 Thread, added  1746  to the table, for card  48232  card #:  110  in file:  trans_thread_0_0.csv
3 Thread, added  3312  to the table, for card  33224  card #:  113  in file:  trans_thread_3_0.csv
10 Thread, added  2577  to the table, for card  98381  card #:  118  in file:  trans_thread_10_0.csv
9 Thread, added  2841  to the table, for card  68557  card #:  114  in file:  trans_thread_9_0.csv
2 Thread, added  7095  to the table, for card  67764  card #:  106  in file:  trans_thread_2_0.csv
1 Thread, added  711  to the table, for card  49292  card #:  104  in file:  trans_thread_1_0.csv
4 Thread, added  838  to the table, for card  104472  card #:  116  in file:  trans_thread_4_0.csv
8 Thread, added  259  to the table, for card  49149  card #:  109  in file:  trans_thread_8_0.csv
6 Thread, added  1209  to the table, for card  97871  card #:  104  in file:  trans_thread_6_0.csv
0 Thread

4 Thread, added  1736  to the table, for card  12275  card #:  127  in file:  trans_thread_4_0.csv
8 Thread, added  1059  to the table, for card  104662  card #:  120  in file:  trans_thread_8_0.csv
0 Thread, added  4276  to the table, for card  76681  card #:  118  in file:  trans_thread_0_0.csv
9 Thread, added  2894  to the table, for card  84774  card #:  124  in file:  trans_thread_9_0.csv
11 Thread, added  1167  to the table, for card  90241  card #:  121  in file:  trans_thread_11_0.csv
7 Thread, added  5284  to the table, for card  31163  card #:  111  in file:  trans_thread_7_0.csv
2021-08-01 19:18:16.490033 2021-08-02 19:18:16.490033
5 Thread, added  8  to the table, for card  115342  card #:  127  in file:  trans_thread_5_0.csv
6 Thread, added  4616  to the table, for card  68472  card #:  1172019-09-07 00:00:00 2019-09-11 00:00:00
2 Thread, added  632  to the table, for card  82457  card #:  119  in file:  trans_thread_2_0.csv
10 Thread, added  3249  to the table, for card  

2 Thread, added  212  to the table, for card  83136  card #:  2780  in file:  trans_thread_2_0.csv
0 Thread, added  45  to the table, for card  63116  card #:  2725  in file:  trans_thread_0_0.csv
3 Thread, added  544  to the table, for card  50892  card #:  2849  in file:  trans_thread_3_0.csv
4 Thread, added  1872  to the table, for card  45032  card #:  2960  in file:  trans_thread_4_0.csv
2020-01-10 00:00:00 2020-01-11 00:00:00
7 Thread, added  756  to the table, for card  25586  card #:  2529  in file:  trans_thread_7_0.csv
8 Thread, added  5952  to the table, for card  10563  card #:  2816  in file:  trans_thread_8_0.csv
10 Thread, added  0  to the table, for card  87535  card #:  3069  in file:  trans_thread_10_0.csv
11 Thread, added  852  to the table, for card  95902  card #:  2862  in file:  trans_thread_11_0.csv
9 Thread, added  147  to the table, for card  51207  card #:  2890  in file:  trans_thread_9_0.csv
0 Thread, added  3023  to the table, for card  67448  card #:  276

4 Thread, added  1438  to the table, for card  112586  card #:  2972  in file:  trans_thread_4_0.csv
8 Thread, added  2956  to the table, for card  91998  card #:  2827  in file:  trans_thread_8_0.csv
7 Thread, added  4206  to the table, for card  30525  card #:  2540  in file:  trans_thread_7_0.csv
5 Thread, added  483  to the table, for card  37883  card #:  2903  in file:  trans_thread_5_0.csv
10 Thread, added  1179  to the table, for card  104779  card #:  3082  in file:  trans_thread_10_0.csv
0 Thread, added  5680  to the table, for card  50160  card #:  2781  in file:  trans_thread_0_0.csv
0 Thread, added  1032  to the table, for card  73807  card #:  2736  in file:  trans_thread_0_0.csv
3 Thread, added  1516  to the table, for card  108668  card #:  2862  in file:  trans_thread_3_0.csv
